In [ ]:
#!/usr/bin/env python3
"""
Greedy Ride Matcher - Python
Matches pending passengers (public_ride_requests) -> riders (riders)
Writes proposals to rider DB collection driver_proposals.

Status Flow:
pending -> accepted -> arrived_at_pickup -> picked_up -> completed

Notes:
- Uses real OTP from passenger requests
- Handles automatic status progression
- Syncs passenger data correctly
"""
import os
import time
import traceback
import random
from math import radians, sin, cos, sqrt, atan2

import firebase_admin
from firebase_admin import credentials, firestore
from firebase_admin.firestore import GeoPoint

# ----------------- CONFIG -----------------
PASSENGER_DB_CREDENTIALS = os.environ.get(
    "PASSENGER_DB_CREDENTIALS",
    "passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json",
)
RIDER_DB_CREDENTIALS = os.environ.get(
    "RIDER_DB_CREDENTIALS",
    "rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json",
)

PASSENGER_REQUESTS_COL = "public_ride_requests"
RIDERS_COL = "riders"
DRIVER_PROPOSALS_COL = "driver_proposals"
COMPLETED_RIDES_COL = "completed_rides"

MAX_MATCH_DISTANCE_KM = float(os.environ.get("MAX_MATCH_DISTANCE_KM", 5.0))

# CRITICAL: Match Flutter app rider statuses
ELIGIBLE_DRIVER_STATUSES = ["available", "on_trip", "idle"]

# ----------------- Utilities -----------------

def log(msg: str):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

def init_firestore_app(cred_path: str, name: str):
    """Initialize Firestore client for a Firebase app."""
    if not os.path.exists(cred_path):
        log(f"ERROR: credential file not found: {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        try:
            app = firebase_admin.get_app(name)
            log(f"Re-using Firebase app '{name}'.")
        except ValueError:
            app = firebase_admin.initialize_app(cred, name=name)
            log(f"Initialized Firebase app '{name}'.")
        return firestore.client(app=app)
    except Exception as e:
        log(f"Error initializing Firebase app '{name}': {e}")
        traceback.print_exc()
        return None

def haversine_km(lat1, lon1, lat2, lon2):
    """Haversine distance between two points in kilometers."""
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, map(float, [lat1, lon1, lat2, lon2]))
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    return 2 * 6371.0 * atan2(sqrt(a), sqrt(1 - a))

def to_geopoint(loc):
    """Normalize input to a Firestore GeoPoint."""
    if loc is None:
        return None
    if isinstance(loc, GeoPoint):
        return loc
    if isinstance(loc, dict):
        lat = next((loc.get(k) for k in ("latitude", "lat", "Latitude") if loc.get(k) is not None), None)
        lon = next((loc.get(k) for k in ("longitude", "lng", "lon", "Longitude") if loc.get(k) is not None), None)
        for cand in ("coords", "location", "geo", "position"):
            if cand in loc and isinstance(loc[cand], dict):
                nested = loc[cand]
                lat = nested.get("latitude") or nested.get("lat") or lat
                lon = nested.get("longitude") or nested.get("lng") or nested.get("lon") or lon
                if lat is not None and lon is not None:
                    break
        if lat is not None and lon is not None:
            try:
                return GeoPoint(float(lat), float(lon))
            except Exception:
                return None
    if hasattr(loc, "latitude") and hasattr(loc, "longitude"):
        try:
            return GeoPoint(float(loc.latitude), float(loc.longitude))
        except Exception:
            return None
    return None

def generate_otp():
    """Generate a 4-digit OTP for ride verification."""
    return str(random.randint(1000, 9999))

# ----------------- Matching Logic -----------------

def create_proposal_payload(passenger_doc, passenger_data, driver_doc_id, driver_data, pickup_dist):
    """Prepare proposal payload with all necessary info for rider."""
    passenger_uid = passenger_data.get("passengerId") or passenger_data.get("passengerUid") or passenger_data.get("riderUid")
    pickup_loc = to_geopoint(passenger_data.get("pickupLocation"))
    dest_loc = to_geopoint(passenger_data.get("destinationLocation"))

    driver_loc = to_geopoint(driver_data.get("currentLocation"))

    # Generate OTP for this ride
    otp = generate_otp()

    # CRITICAL: This payload MUST match Flutter app field names EXACTLY
    payload = {
        # --- Core Identification ---
        "request_id": passenger_doc.id,
        "status": "pending_acceptance",  # Flutter queries: where('status', isEqualTo: 'pending_acceptance')
        
        # --- PASSENGER Info (CRITICAL: Use passenger fields) ---
        "passengerId": passenger_data.get("passengerId") or passenger_uid,
        "passengerName": passenger_data.get("passengerName") or "Passenger",
        "passengerPhone": passenger_data.get("passengerPhone") or "N/A",
        "passengerRating": passenger_data.get("passengerRating") or 5.0,
        
        # --- Location Fields ---
        "pickupLocation": pickup_loc,
        "destinationLocation": dest_loc,
        "pickupAddress": passenger_data.get("pickupAddress") or passenger_data.get("pickup_address") or "Pickup Location",
        "destinationAddress": passenger_data.get("destinationAddress") or passenger_data.get("destination_address") or "Destination Location",
        
        # --- Rider/Driver Reference ---
        "riderUid": driver_doc_id,  # FLUTTER QUERIES: where('riderUid', isEqualTo: user.uid)
        
        # --- Driver Info for Display ---
        "riderName": driver_data.get("riderName") or "Unknown Driver",
        "riderPhone": driver_data.get("riderPhone") or "Not Provided",
        "driverVehicle": driver_data.get("vehicleType") or "Unknown Vehicle",
        "riderLocation": driver_loc,
        "riderRating": driver_data.get("riderRating") or 5.0,
        
        # --- OTP and Verification ---
        "otp": otp,  # Real OTP for verification
        "otpVerified": False,
        
        # --- Timestamps ---
        "createdAt": firestore.SERVER_TIMESTAMP,
        "lastUpdated": firestore.SERVER_TIMESTAMP,
        "requestTimestamp": passenger_data.get("requestTimestamp") or firestore.SERVER_TIMESTAMP,
        "acceptedTimestamp": None,
        "arrivalTimestamp": None,
        "pickupTimestamp": None,
        "completionTimestamp": None,
        "cancellationTimestamp": None,
        
        # --- Additional Fields ---
        "fareAmount": passenger_data.get("fareAmount") or 0.0,
        "paymentMethod": passenger_data.get("paymentMethod") or "Cash",
        "rideType": passenger_data.get("rideType") or "Standard",
        "passengerCount": passenger_data.get("passengerCount") or 1,
        "luggageCount": passenger_data.get("luggageCount") or 0,
        "vehiclePreference": passenger_data.get("vehiclePreference") or "Any",
        "specialRequests": passenger_data.get("specialRequests") or "None",
        
        # --- Distance and Duration ---
        "estimatedDistance": passenger_data.get("estimatedDistance") or "0 km",
        "estimatedDuration": passenger_data.get("estimatedDuration") or "0 min",
        "estimatedPickupTime": passenger_data.get("estimatedPickupTime") or "0 min",
        "distanceToPickup": pickup_dist,
        "match_score": 1.0 / (1.0 + pickup_dist) if pickup_dist else 0.5,
        
        # --- Ride Status Tracking ---
        "hasArrivedAtPickupPoint": False,
        "hasArrivedAtDestination": False,
        "passengerPickedUp": False,
    }

    return payload

def try_reserve_driver_and_create_proposal(rider_db, passenger_db, passenger_doc, driver_doc_id, driver_data, proposal_payload):
    """Reserve driver and write proposal atomically, then update passenger doc."""
    rider_ref = rider_db.collection(RIDERS_COL).document(driver_doc_id)
    proposal_ref = rider_db.collection(DRIVER_PROPOSALS_COL).document()

    @firestore.transactional
    def txn_reserve(transaction):
        snapshot = rider_ref.get(transaction=transaction)
        if not snapshot.exists:
            raise RuntimeError("Driver doc disappeared during reservation.")
        current_status = snapshot.get("status")
        if current_status not in ELIGIBLE_DRIVER_STATUSES:
            raise RuntimeError(f"Driver {driver_doc_id} status '{current_status}' not eligible.")
        # Reserve the driver
        transaction.update(rider_ref, {
            "status": "reserved_for_proposal", 
            "reserved_for_request": passenger_doc.id
        })
        # Create the proposal
        transaction.set(proposal_ref, proposal_payload)

    try:
        transaction = rider_db.transaction()
        txn_reserve(transaction)
        log(f"✅ Successfully reserved driver {driver_doc_id} and created proposal")
    except Exception as e:
        log(f"❌ Failed to reserve driver {driver_doc_id}: {e}")
        traceback.print_exc()
        return False, None

    try:
        # Update passenger document to show it's been proposed
        passenger_update = {
            "status": "proposed",
            "riderUid": driver_doc_id,
            "riderName": driver_data.get("riderName") or "Unknown Driver",
            "riderPhone": driver_data.get("riderPhone") or "Not Provided",
            "proposed_at": firestore.SERVER_TIMESTAMP,
            "proposal_id": proposal_ref.id,
            "assigned_otp": proposal_payload["otp"],  # Store OTP in passenger doc too
        }
        passenger_db.collection(PASSENGER_REQUESTS_COL).document(passenger_doc.id).update(passenger_update)
        log(f"✅ Passenger {passenger_doc.id} updated to 'proposed' with OTP: {proposal_payload['otp']}")
        return True, proposal_ref.id
    except Exception as e:
        log(f"❌ Failed to update passenger {passenger_doc.id}: {e}")
        traceback.print_exc()
        # Revert rider status on failure
        try:
            rider_db.collection(RIDERS_COL).document(driver_doc_id).update({
                "status": driver_data.get("status", "available"),
                "reserved_for_request": firestore.DELETE_FIELD
            })
            log(f"✅ Reverted rider {driver_doc_id} status back to {driver_data.get('status', 'available')}")
        except Exception:
            log(f"⚠️ Warning: failed to revert reservation for driver {driver_doc_id}")
        return False, None

def match_one_request(passenger_db, rider_db, passenger_doc):
    """Match a single passenger request to the best available driver."""
    try:
        passenger_data = passenger_doc.to_dict() or {}
        current_status = passenger_data.get("status", "").lower()
        
        # Only match pending/searching requests
        if not passenger_data or current_status not in ("pending", "searching"):
            log(f"⚠️ Passenger {passenger_doc.id} status '{current_status}' not eligible for matching")
            return

        pickup = to_geopoint(passenger_data.get("pickupLocation"))
        dest = to_geopoint(passenger_data.get("destinationLocation"))
        
        if not (pickup and dest):
            log(f"❌ Passenger {passenger_doc.id}: pickup/destination invalid - skipping.")
            return

        log(f"🔍 Matching passenger {passenger_doc.id} at ({pickup.latitude:.6f},{pickup.longitude:.6f})")

        # Get all available riders
        riders_ref = rider_db.collection(RIDERS_COL)
        riders = list(riders_ref.stream())
        
        if not riders:
            log("❌ No riders found in database")
            return
# ---- GREEDY SELECTION STEP ----
# Select the closest available driver for this request
        best_driver = None
        best_score = float("inf")
        best_pickup_dist = float("inf")

        eligible_count = 0
        
        for rdoc in riders:
            try:
                rdata = rdoc.to_dict() or {}
                driver_id = rdoc.id
                
                # Check if rider is available (Flutter uses 'available' status)
                rider_status = rdata.get("status", "").lower()
                if rider_status != "available":
                    continue
                    
                eligible_count += 1
                
                # Get rider's current location
                driver_loc = to_geopoint(rdata.get("currentLocation"))
                if not driver_loc:
                    continue

                # Calculate distance from rider to pickup
                pickup_dist = haversine_km(
                    driver_loc.latitude, driver_loc.longitude,
                    pickup.latitude, pickup.longitude
                )
                
                # Check if within maximum distance
                if pickup_dist > MAX_MATCH_DISTANCE_KM:
                    continue

                # Simple scoring: prefer closer drivers
                score = pickup_dist
                
                if score < best_score or (score == best_score and pickup_dist < best_pickup_dist):
                    best_score = score
                    best_driver = (driver_id, rdata, driver_loc, pickup_dist)
                    best_pickup_dist = pickup_dist
                    
            except Exception as e:
                log(f"⚠️ Error evaluating driver {rdoc.id}: {e}")
                continue

        log(f"📊 Found {eligible_count} eligible riders out of {len(riders)} total riders")

        if not best_driver:
            log(f"❌ No suitable driver found for passenger {passenger_doc.id}")
            return

        driver_doc_id, driver_data, driver_loc, pickup_dist = best_driver
        log(f"✅ Selected driver {driver_doc_id}: distance={pickup_dist:.3f} km")

        # Create proposal with all necessary data
        proposal_payload = create_proposal_payload(
            passenger_doc, passenger_data, driver_doc_id, driver_data, pickup_dist
        )
        
        success, proposal_id = try_reserve_driver_and_create_proposal(
            rider_db, passenger_db, passenger_doc, driver_doc_id, driver_data, proposal_payload
        )

        if success:
            log(f"🎉 Proposal created (id={proposal_id}) for passenger {passenger_doc.id} -> rider {driver_doc_id}")
            log(f"📋 OTP for this ride: {proposal_payload['otp']}")
            
            # Verify the proposal was created with correct passenger fields
            try:
                created_proposal = rider_db.collection(DRIVER_PROPOSALS_COL).document(proposal_id).get()
                if created_proposal.exists:
                    proposal_data = created_proposal.to_dict()
                    log(f"📋 Created proposal with passenger: {proposal_data.get('passengerName')} ({proposal_data.get('passengerPhone')})")
            except Exception as e:
                log(f"⚠️ Could not verify created proposal: {e}")
                
        else:
            log(f"❌ Failed to create proposal for passenger {passenger_doc.id}")

    except Exception as e:
        log(f"💥 Error matching passenger {passenger_doc.id}: {e}")
        traceback.print_exc()

# ----------------- Status mapping helpers -----------------

def map_proposal_status_to_passenger(proposal_status):
    """Map a driver_proposal.status to a passenger request status."""
    ps = (proposal_status or "").lower()
    status_map = {
        "accepted": "accepted",
        "arrived_at_pickup": "arrived_at_pickup", 
        "picked_up": "picked_up",
        "completed": "completed",
        "rejected": "rejected",
        "cancelled": "cancelled",
        "cancelled_by_rider": "cancelled",
        "cancelled_by_passenger": "cancelled",
    }
    return status_map.get(ps)

# ----------------- Driver Proposal Listener (progress updates) -----------------

def listen_for_driver_proposal_progress(rider_db, passenger_db):
    """Listen for updates on driver proposals and propagate to passenger request statuses."""
    interesting_statuses = [
        "accepted", "arrived_at_pickup", "picked_up", 
        "completed", "rejected", "cancelled", "cancelled_by_rider"
    ]

    def on_proposals_snapshot(col_snapshot, changes, read_time):
        for change in changes:
            try:
                if change.type.name not in ("ADDED", "MODIFIED"):
                    continue

                doc = change.document
                data = doc.to_dict() or {}
                proposal_status = data.get("status")
                request_id = data.get("request_id")
                rider_uid = data.get("riderUid")

                if not request_id:
                    continue

                log(f"🔄 Proposal {doc.id} status changed to: {proposal_status}")

                # Update passenger document
                pdoc_ref = passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id)
                
                try:
                    pdoc = pdoc_ref.get()
                    if not pdoc.exists:
                        log(f"⚠️ Passenger document {request_id} not found")
                        continue
                except Exception as e:
                    log(f"❌ Error fetching passenger doc {request_id}: {e}")
                    continue

                mapped_status = map_proposal_status_to_passenger(proposal_status)
                if not mapped_status:
                    continue

                update_data = {"status": mapped_status}
                
                # Add timestamp based on status
                if mapped_status == "accepted":
                    update_data["accepted_at"] = firestore.SERVER_TIMESTAMP
                elif mapped_status == "arrived_at_pickup":
                    update_data["arrived_at"] = firestore.SERVER_TIMESTAMP
                elif mapped_status == "picked_up":
                    update_data["picked_up_at"] = firestore.SERVER_TIMESTAMP
                elif mapped_status == "completed":
                    update_data["completed_at"] = firestore.SERVER_TIMESTAMP
                    # Store completed ride in history
                    _store_completed_ride(rider_db, passenger_db, doc, data)

                # Update passenger document
                try:
                    pdoc_ref.update(update_data)
                    log(f"✅ Updated passenger {request_id} status to: {mapped_status}")
                except Exception as e:
                    log(f"❌ Failed to update passenger {request_id}: {e}")

                # Update rider status if needed
                if rider_uid and mapped_status in ["accepted", "completed", "rejected", "cancelled"]:
                    try:
                        rider_update = {}
                        if mapped_status == "accepted":
                            rider_update["status"] = "on_trip"
                        elif mapped_status in ["completed", "rejected", "cancelled"]:
                            rider_update["status"] = "available"
                            rider_update["reserved_for_request"] = firestore.DELETE_FIELD
                        
                        if rider_update:
                            rider_db.collection(RIDERS_COL).document(rider_uid).update(rider_update)
                            log(f"✅ Updated rider {rider_uid} status to {rider_update.get('status')}")
                    except Exception as e:
                        log(f"❌ Failed to update rider {rider_uid}: {e}")

            except Exception as e:
                log(f"💥 Error processing proposal update: {e}")
                traceback.print_exc()

    try:
        query = rider_db.collection(DRIVER_PROPOSALS_COL)
        query.on_snapshot(on_proposals_snapshot)
        log("✅ Driver proposal progress listener attached")
    except Exception as e:
        log(f"❌ Failed to attach driver proposal progress listener: {e}")

def _store_completed_ride(rider_db, passenger_db, proposal_doc, proposal_data):
    """Store completed ride in history collection."""
    try:
        request_id = proposal_data.get("request_id")
        if not request_id:
            return

        # Get passenger data for history
        passenger_doc = passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).get()
        passenger_data = passenger_doc.to_dict() or {} if passenger_doc.exists else {}

        completed_ride_data = {
            # Ride identification
            "proposal_id": proposal_doc.id,
            "request_id": request_id,
            
            # Passenger info
            "passengerId": proposal_data.get("passengerId"),
            "passengerName": proposal_data.get("passengerName"),
            "passengerPhone": proposal_data.get("passengerPhone"),
            "passengerRating": proposal_data.get("passengerRating"),
            
            # Driver info
            "riderUid": proposal_data.get("riderUid"),
            "riderName": proposal_data.get("riderName"),
            "riderPhone": proposal_data.get("riderPhone"),
            "riderRating": proposal_data.get("riderRating"),
            
            # Ride details
            "pickupLocation": proposal_data.get("pickupLocation"),
            "destinationLocation": proposal_data.get("destinationLocation"),
            "pickupAddress": proposal_data.get("pickupAddress"),
            "destinationAddress": proposal_data.get("destinationAddress"),
            
            # Fare and payment
            "fareAmount": proposal_data.get("fareAmount"),
            "paymentMethod": proposal_data.get("paymentMethod"),
            "rideType": proposal_data.get("rideType"),
            
            # Timestamps
            "requestTime": proposal_data.get("requestTimestamp"),
            "acceptedTime": proposal_data.get("acceptedTimestamp"),
            "arrivalTime": proposal_data.get("arrivalTimestamp"),
            "pickupTime": proposal_data.get("pickupTimestamp"),
            "completionTime": proposal_data.get("completionTimestamp"),
            "completedAt": firestore.SERVER_TIMESTAMP,
            
            # Ride metrics
            "estimatedDistance": proposal_data.get("estimatedDistance"),
            "estimatedDuration": proposal_data.get("estimatedDuration"),
            "distanceToPickup": proposal_data.get("distanceToPickup"),
            
            # Additional info
            "passengerCount": proposal_data.get("passengerCount"),
            "luggageCount": proposal_data.get("luggageCount"),
            "vehiclePreference": proposal_data.get("vehiclePreference"),
            "specialRequests": proposal_data.get("specialRequests"),
            
            # Status
            "status": "completed",
        }

        # Store in completed rides collection
        rider_db.collection(COMPLETED_RIDES_COL).add(completed_ride_data)
        log(f"✅ Completed ride stored in history: {request_id}")

    except Exception as e:
        log(f"❌ Error storing completed ride: {e}")

# ----------------- Rider Location Updates -----------------

def listen_for_rider_location_updates(rider_db, passenger_db):
    """Listen for rider location updates and update passenger request in real-time."""
    def on_rider_location_snapshot(col_snapshot, changes, read_time):
        for change in changes:
            try:
                if change.type.name not in ("MODIFIED", "ADDED"):
                    continue

                doc = change.document
                data = doc.to_dict() or {}
                rider_uid = doc.id
                current_location = data.get("currentLocation")
                current_ride_request = data.get("reserved_for_request")

                if not current_location or not current_ride_request:
                    continue

                # Update passenger request with rider location
                try:
                    pdoc_ref = passenger_db.collection(PASSENGER_REQUESTS_COL).document(current_ride_request)
                    pdoc_ref.update({
                        "riderLocation": current_location,
                        "lastLocationUpdate": firestore.SERVER_TIMESTAMP,
                    })
                    log(f"📍 Updated rider location for request {current_ride_request}")
                except Exception as e:
                    log(f"❌ Failed to update passenger location: {e}")

            except Exception as e:
                log(f"💥 Error processing rider location update: {e}")

    try:
        query = rider_db.collection(RIDERS_COL)
        query.on_snapshot(on_rider_location_snapshot)
        log("✅ Rider location update listener attached")
    except Exception as e:
        log(f"❌ Failed to attach rider location listener: {e}")

# ----------------- Passenger Data Sync -----------------

def sync_passenger_data_to_proposals(rider_db, passenger_db):
    """Ensure driver_proposals have correct passenger data by syncing from passenger DB."""
    def on_passenger_snapshot(col_snapshot, changes, read_time):
        for change in changes:
            try:
                if change.type.name not in ("ADDED", "MODIFIED"):
                    continue

                passenger_doc = change.document
                passenger_data = passenger_doc.to_dict() or {}
                
                # Check if this passenger has a proposal in rider DB
                proposals_ref = rider_db.collection(DRIVER_PROPOSALS_COL)
                proposals_query = proposals_ref.where("request_id", "==", passenger_doc.id).stream()
                
                for proposal_doc in proposals_query:
                    proposal_data = proposal_doc.to_dict() or {}
                    
                    # Check if passenger data needs updating in proposal
                    needs_update = False
                    update_data = {}
                    
                    # Compare passenger fields
                    if passenger_data.get("passengerName") != proposal_data.get("passengerName"):
                        update_data["passengerName"] = passenger_data.get("passengerName")
                        needs_update = True
                    
                    if passenger_data.get("passengerPhone") != proposal_data.get("passengerPhone"):
                        update_data["passengerPhone"] = passenger_data.get("passengerPhone")
                        needs_update = True
                    
                    if passenger_data.get("passengerId") != proposal_data.get("passengerId"):
                        update_data["passengerId"] = passenger_data.get("passengerId")
                        needs_update = True
                    
                    if needs_update:
                        try:
                            rider_db.collection(DRIVER_PROPOSALS_COL).document(proposal_doc.id).update(update_data)
                            log(f"✅ Synced passenger data to proposal {proposal_doc.id}: {update_data}")
                        except Exception as e:
                            log(f"❌ Failed to sync passenger data to proposal {proposal_doc.id}: {e}")
                            
            except Exception as e:
                log(f"💥 Error in passenger sync listener: {e}")
                traceback.print_exc()

    try:
        query = passenger_db.collection(PASSENGER_REQUESTS_COL)
        query.on_snapshot(on_passenger_snapshot)
        log("✅ Passenger data sync listener attached")
    except Exception as e:
        log(f"❌ Failed to attach passenger data sync listener: {e}")

# ----------------- Firestore Listener for Pending Requests -----------------

def on_pending_requests_snapshot(col_snapshot, changes, read_time):
    """Firestore listener callback for new or modified pending requests."""
    for change in changes:
        try:
            if change.type.name in ("ADDED", "MODIFIED"):
                doc = change.document
                data = doc.to_dict() or {}
                current_status = data.get("status", "").lower()
                
                if current_status in ("pending", "searching"):
                    log(f"🎯 Processing {current_status} passenger request: {doc.id}")
                    match_one_request(db_passenger, db_rider, doc)
                    
        except Exception as e:
            log(f"💥 Error in snapshot handler: {e}")
            traceback.print_exc()

# ----------------- Main -----------------

if __name__ == "__main__":
    db_passenger = init_firestore_app(PASSENGER_DB_CREDENTIALS, "passenger_app")
    db_rider = init_firestore_app(RIDER_DB_CREDENTIALS, "rider_app")

    if not db_passenger or not db_rider:
        log("💥 FATAL: Firestore clients failed to initialize. Exiting.")
        raise SystemExit(1)

    log("🚀 Greedy Ride Matcher Started Successfully!")
    log(f"📡 Listening to: {PASSENGER_REQUESTS_COL} (passenger requests)")
    log(f"💾 Writing to: {DRIVER_PROPOSALS_COL} (rider proposals)")
    log(f"📊 Status Flow: pending -> accepted -> arrived_at_pickup -> picked_up -> completed")
    log(f"🔐 OTP Verification: Enabled with real 4-digit OTPs")

    try:
        # Start listener for pending passenger requests
        query = db_passenger.collection(PASSENGER_REQUESTS_COL)
        query.on_snapshot(on_pending_requests_snapshot)

        # Listen for driver proposal progress
        listen_for_driver_proposal_progress(db_rider, db_passenger)

        # Listen for rider location updates
        listen_for_rider_location_updates(db_rider, db_passenger)

        # Sync passenger data to proposals
        sync_passenger_data_to_proposals(db_rider, db_passenger)

        log("✅ All listeners attached and running... (Ctrl+C to stop)")

        # Keep the program running
        while True:
            time.sleep(10)  # Sleep longer to reduce CPU usage
            log("💓 Matcher heartbeat...")
            
    except KeyboardInterrupt:
        log("🛑 Shutting down (KeyboardInterrupt).")
    except Exception as e:
        log(f"💥 FATAL error in main loop: {e}")
        traceback.print_exc()

[2026-01-08 10:46:49] Initialized Firebase app 'passenger_app'.
[2026-01-08 10:46:49] Initialized Firebase app 'rider_app'.
[2026-01-08 10:46:49] 🚀 Greedy Ride Matcher Started Successfully!
[2026-01-08 10:46:49] 📡 Listening to: public_ride_requests (passenger requests)
[2026-01-08 10:46:49] 💾 Writing to: driver_proposals (rider proposals)
[2026-01-08 10:46:49] 📊 Status Flow: pending -> accepted -> arrived_at_pickup -> picked_up -> completed
[2026-01-08 10:46:49] 🔐 OTP Verification: Enabled with real 4-digit OTPs
[2026-01-08 10:46:49] ✅ Driver proposal progress listener attached
[2026-01-08 10:46:49] ✅ Rider location update listener attached
[2026-01-08 10:46:49] ✅ Passenger data sync listener attached
[2026-01-08 10:46:49] ✅ All listeners attached and running... (Ctrl+C to stop)
[2026-01-08 10:46:55] 🔄 Proposal 32Vw5JKYolN9xwk5amwl status changed to: picked_up


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[2026-01-08 10:46:59] 💓 Matcher heartbeat...
[2026-01-08 10:47:01] 📍 Updated rider location for request AGMaeSGxLPNcX52pzybH
[2026-01-08 10:47:01] ✅ Updated passenger How2mSLWCb6F2B6VmGnD status to: picked_up
[2026-01-08 10:47:01] 🔄 Proposal 3s4DQlDIbzcvIsY2GI6t status changed to: arrived_at_pickup
[2026-01-08 10:47:09] 💓 Matcher heartbeat...
[2026-01-08 10:47:09] ✅ Updated passenger hn6QFMzgQZiBMcJrGOdI status to: arrived_at_pickup
[2026-01-08 10:47:09] 🔄 Proposal 5p6dhiX2FNevLqR9HUEV status changed to: completed
[2026-01-08 10:47:11] ✅ Completed ride stored in history: NXlTRquZXnDypucem1eB
[2026-01-08 10:47:11] ✅ Updated passenger NXlTRquZXnDypucem1eB status to: completed
[2026-01-08 10:47:12] ✅ Updated rider a8u9W0osH4bbzvvmPUbCV2pManf2 status to available
[2026-01-08 10:47:12] 🔄 Proposal C8RHFuQzTcOnoeA5h9gm status changed to: completed
[2026-01-08 10:47:13] ✅ Completed ride stored in history: 2VT4vWlOBWtlflsv6hxE
[2026-01-08 10:47:13] ✅ Updated passenger 2VT4vWlOBWtlflsv6hxE statu